In [1]:
import numpy as np
import numpy
import pyscf
import pyscf.gto
import pyscf.qmmm
import pyscf.scf
import pyscf.dft
import pyscf.lib
from pyscf.data import nist
import qml

angstrom = 1 / 0.52917721067

In [2]:
com = qml.Compound(xyz='/home/misa/datasets/qm9/dsgdb9nsd_001212.xyz')
total_elec = com.nuclear_charges.sum()
num_elecs = [22, 30, 38, 44, 52]
alchemical_potentials = []
e_homo = []
e_lumo = []

for num_elec in num_elecs:
    print(f'Calculating {num_elec}')
    lam_val = num_elec/total_elec
    # core_elec = com.nuclear_charges.sum() - total_num_ve

    mol = pyscf.gto.Mole()
    for ch, coords in zip(com.nuclear_charges, com.coordinates):
        mol.atom.append([ch, coords])

    mol.basis = 'def2-tzvp'
    print(total_elec - num_elec)
    mol.charge = total_elec - num_elec
    mol.verbose = 4
    mol.build()

    method = "HF"
    if method not in ["CCSD", "HF"]:
        raise NotImplementedError("Method %s not supported." % method)

    deltaZ = -com.nuclear_charges*(1-lam_val)
    includeonly = np.arange(len(mol.atom_coords()))


    def add_qmmm(calc, mol, deltaZ):
        mf = pyscf.qmmm.mm_charge(calc, mol.atom_coords()[includeonly]/ angstrom, deltaZ)

        def energy_nuc(self):
            q = mol.atom_charges().astype(np.float)
            q[includeonly] += deltaZ
            return mol.energy_nuc(q)

        mf.energy_nuc = energy_nuc.__get__(mf, mf.__class__)

        return mf

    if method == "HF":
        calc = add_qmmm(pyscf.scf.RHF(mol), mol, deltaZ)
        hfe = calc.kernel(verbose=0)
        dm1_ao = calc.make_rdm1()
        total_energy = calc.e_tot
    if method == "CCSD":
        calc = add_qmmm(pyscf.scf.RHF(mol), mol, deltaZ)
        hfe = calc.kernel(verbose=0)
        mycc = pyscf.cc.CCSD(calc).run()
        dm1 = mycc.make_rdm1()
        dm1_ao = np.einsum("pi,ij,qj->pq", calc.mo_coeff, dm1, calc.mo_coeff.conj())
        total_energy = mycc.e_tot

    # GRIDLESS, as things should be ############################
    # Total energy of SCF run

    print("TOTAL_ENERGY", total_energy)

    # Electronic EPN from electron density
    alch_pot_tmp = []
    for site in includeonly:
        mol.set_rinv_orig_(mol.atom_coords()[site])
#         print("ELECTRONIC_EPN", site, np.matmul(dm1_ao, mol.intor("int1e_rinv")).trace())
        alch_pot_tmp.append(np.matmul(dm1_ao, mol.intor("int1e_rinv")).trace())
    alchemical_potentials.append(alch_pot_tmp)
    
    # energy of HOMO and LUMO
    nocc = int((total_elec - mol.charge)/2)
    e_homo.append(calc.mo_energy[nocc-1])
    e_lumo.append(calc.mo_energy[nocc])

Calculating 22
30
System: uname_result(system='Linux', node='pc-avl24', release='4.15.0-106-generic', version='#107-Ubuntu SMP Thu Jun 4 11:27:52 UTC 2020', machine='x86_64', processor='x86_64')  Threads 8
Python 3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]
numpy 1.16.4  scipy 1.2.1
Date: Wed Jun 24 18:05:43 2020
PySCF version 1.7.1
PySCF path  /home/misa/anaconda3/envs/atomic-energies/lib/python3.7/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 12
[INPUT] num. electrons = 22
[INPUT] charge = 30
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT]  1 C     -0.136883959200   1.448579793000   0.011368940800 AA   -0.258673193734   2.737419078349   0.021484184438 Bohr
[INPUT]  2 C      0.064357729200  -0.043732595200   0.001817971000 AA    0.121618482187  -0.082642627644   0.003435467292 Bohr
[INPUT]  3 O      1.114424575900  -0.629340888600  -0.012303871000 AA    2.105957234936  -

Overwritten attributes  energy_nuc  of <class 'pyscf.qmmm.itrf.qmmm_for_scf.<locals>.QMMM'>


init E= 217.021850274313
  HOMO = 4.99480993985173  LUMO = 5.05535175096831


/home/misa/anaconda3/envs/atomic-energies/lib/python3.7/site-packages/pyscf/gto/mole.py:1047: UserWarning: Function mol.dumps drops attribute charge because it is not JSON-serializable
  warnings.warn(msg)


cycle= 1 E= 18.2295129366766  delta_E= -199  |g|=    3  |ddm|= 2.93e+03
  HOMO = -3.20597898416894  LUMO = -2.90640058424913
cycle= 2 E= -34.4280364571358  delta_E= -52.7  |g|= 6.19  |ddm|= 2.92e+03
  HOMO = -2.98896083270616  LUMO = -2.69471582403049
cycle= 3 E= -35.0933217479009  delta_E= -0.665  |g|=  5.9  |ddm|=  0.6
  HOMO = -2.91664527796784  LUMO = -2.61322119060863
cycle= 4 E= -35.3506091790106  delta_E= -0.257  |g|=  5.8  |ddm|= 0.288
  HOMO = -2.91445012278743  LUMO = -2.60716447233939
cycle= 5 E= -35.3386223839958  delta_E= 0.012  |g|=  5.8  |ddm|= 0.0373
  HOMO = -2.88195731288167  LUMO = -2.57739633851852
cycle= 6 E= -35.4301342407188  delta_E= -0.0915  |g|= 5.75  |ddm|= 0.175
  HOMO = -2.8826136949189  LUMO = -2.57822432525167
cycle= 7 E= -35.4266225877024  delta_E= 0.00351  |g|= 5.76  |ddm|= 0.00842
  HOMO = -2.83310372526945  LUMO = -2.52961464479686
cycle= 8 E= -35.5768231790075  delta_E= -0.15  |g|= 5.68  |ddm|= 0.163
  HOMO = -2.83421400573287  LUMO = -2.530464632046

cycle= 11 E= -95.7521268456508  delta_E= -0.000629  |g|= 0.00769  |ddm|= 0.289
  HOMO = -0.256456900036965  LUMO = -0.0654871767105219
cycle= 12 E= -95.7522200796741  delta_E= -9.32e-05  |g|= 0.00293  |ddm|= 0.187
  HOMO = -0.256646914489457  LUMO = -0.0654246859400469
cycle= 13 E= -95.7522362966277  delta_E= -1.62e-05  |g|= 0.00263  |ddm|= 0.0458
  HOMO = -0.256708848726039  LUMO = -0.0652944026996572
cycle= 14 E= -95.7522483967379  delta_E= -1.21e-05  |g|= 0.00196  |ddm|= 0.0304
  HOMO = -0.256927080629721  LUMO = -0.0651968500983719
cycle= 15 E= -95.7522647294249  delta_E= -1.63e-05  |g|= 0.00114  |ddm|= 0.0376
  HOMO = -0.257011084104842  LUMO = -0.0651783507922903
cycle= 16 E= -95.7522718311548  delta_E= -7.1e-06  |g|= 0.00105  |ddm|= 0.0196
  HOMO = -0.257247833331966  LUMO = -0.0652550008197147
cycle= 17 E= -95.7522868956679  delta_E= -1.51e-05  |g|= 0.000793  |ddm|= 0.054
  HOMO = -0.257382742049432  LUMO = -0.0652227595432592
cycle= 18 E= -95.7522937134919  delta_E= -6.82e-06 

cycle= 16 E= -169.200443574806  delta_E= -0.0244  |g|= 0.0913  |ddm|= 0.447
  HOMO = -0.309301398874416  LUMO = -0.0814625588856797
cycle= 17 E= -169.208882790397  delta_E= -0.00844  |g|= 0.0439  |ddm|= 0.306
  HOMO = -0.309792114802556  LUMO = -0.0814331070608619
cycle= 18 E= -169.21095648375  delta_E= -0.00207  |g|= 0.0204  |ddm|= 0.157
  HOMO = -0.312548457773003  LUMO = -0.0823176461461332
cycle= 19 E= -169.211556404794  delta_E= -0.0006  |g|= 0.0096  |ddm|= 0.0894
  HOMO = -0.311971010080278  LUMO = -0.0817779838972892
cycle= 20 E= -169.211728008532  delta_E= -0.000172  |g|= 0.00472  |ddm|= 0.0464
  HOMO = -0.312159311869928  LUMO = -0.0816501436299966
cycle= 21 E= -169.211779581168  delta_E= -5.16e-05  |g|= 0.00292  |ddm|= 0.0244
  HOMO = -0.312210068657246  LUMO = -0.0819465617179587
cycle= 22 E= -169.211803218424  delta_E= -2.36e-05  |g|= 0.00179  |ddm|= 0.0187
  HOMO = -0.312129741480624  LUMO = -0.0818687649613929
cycle= 23 E= -169.211814457227  delta_E= -1.12e-05  |g|= 0.001

  HOMO = -0.378167702099691  LUMO = -0.174307584593061
cycle= 10 E= -240.439417245706  delta_E= -0.165  |g|= 0.392  |ddm|= 0.751
  HOMO = -0.393307257400346  LUMO = -0.176581095488569
cycle= 11 E= -240.478733127181  delta_E= -0.0393  |g|= 0.32  |ddm|= 0.259
  HOMO = -0.394614309307799  LUMO = -0.165655346881679
cycle= 12 E= -240.508706812487  delta_E= -0.03  |g|= 0.229  |ddm|= 0.323
  HOMO = -0.376753970555236  LUMO = -0.16776941145483
cycle= 13 E= -240.534767758626  delta_E= -0.0261  |g|= 0.121  |ddm|= 0.676
  HOMO = -0.369015950733198  LUMO = -0.170681891501021
cycle= 14 E= -240.543743465236  delta_E= -0.00898  |g|= 0.0484  |ddm|= 0.449
  HOMO = -0.367054775071167  LUMO = -0.16914094712569
cycle= 15 E= -240.546060776088  delta_E= -0.00232  |g|= 0.018  |ddm|= 0.202
  HOMO = -0.367872909445073  LUMO = -0.166485518391538
cycle= 16 E= -240.546605990708  delta_E= -0.000545  |g|= 0.0103  |ddm|= 0.0722
  HOMO = -0.370675489970641  LUMO = -0.165510141199804
cycle= 17 E= -240.546833374011  de

  HOMO = -0.47001553116705  LUMO = 0.117421266573756
cycle= 10 E= -358.701270379053  delta_E= -4.97e-08  |g|= 8.39e-05  |ddm|= 0.00062
  HOMO = -0.470014715741997  LUMO = 0.117422711744735
cycle= 11 E= -358.701270383211  delta_E= -4.16e-09  |g|= 2.76e-05  |ddm|= 0.000189
  HOMO = -0.470018382935321  LUMO = 0.117421290362443
cycle= 12 E= -358.701270383615  delta_E= -4.04e-10  |g|= 1.04e-05  |ddm|= 4.6e-05
  HOMO = -0.470018013000782  LUMO = 0.117421878411869
Extra cycle  E= -358.701270383675  delta_E= -6.05e-11  |g|= 7.28e-06  |ddm|= 1.18e-05
converged SCF energy = -358.701270383675
TOTAL_ENERGY -358.7012703836752


In [3]:
e_homo

[-0.17491144565201802,
 -0.25773431164138594,
 -0.312081098289783,
 -0.3721937061368556,
 -0.47001801300078155]

In [4]:
e_lumo

[-0.024839642886915054,
 -0.0652063328553047,
 -0.0817593672596734,
 -0.16351273977697436,
 0.11742187841186912]

In [5]:
alchemical_potentials

[[8.531093768052123,
  9.808937622579315,
  10.600875496679217,
  11.388630794093007,
  9.428918850383681,
  8.834636341757108,
  8.674870007518825,
  4.52504049030395,
  4.492587188847226,
  4.492599344087726,
  5.3083095032944225,
  5.30818047655433],
 [12.48194451044757,
  14.371666784313643,
  15.61795994501552,
  16.76601069887471,
  13.825993238871046,
  12.962821713640196,
  13.013380309423802,
  6.0420513680497425,
  6.279591457053345,
  6.27962477134309,
  7.298101899492796,
  7.297926549288784],
 [16.72787267142113,
  19.185904108729325,
  21.26520211305058,
  22.520261329232866,
  18.328138815739827,
  17.30683017816755,
  17.541220494415548,
  7.9785276758931385,
  8.232502563338581,
  8.232783279927503,
  9.32188454757556,
  9.32140826233974],
 [19.830840761082353,
  22.59703295721712,
  25.531875307553378,
  26.976480786404032,
  21.707134683008153,
  20.826663564791893,
  21.28876228769513,
  9.117925063533697,
  9.619480127682596,
  9.61970949354337,
  10.85656526159121

In [ ]:
# com = qml.Compound(xyz='/home/misa/datasets/qm9/dsgdb9nsd_001212.xyz')
# total_elec = com.nuclear_charges.sum()
# total_num_ve = 38
# num_ves = [8, 16, 24, 30, 38]
# alchemical_potentials = []
# e_homo = []
# e_lumo = []

# for num_ve in num_ves:
#     print(f'Calculating {num_ve}')
#     lam_val = num_ve/total_num_ve
#     # core_elec = com.nuclear_charges.sum() - total_num_ve

#     mol = pyscf.gto.Mole()
#     for ch, coords in zip(com.nuclear_charges, com.coordinates):
#         mol.atom.append([ch, coords])

#     mol.basis = 'def2-tzvp'
#     mol.charge = total_num_ve - num_ve
#     mol.verbose = 0
#     mol.build()

#     method = "HF"
#     if method not in ["CCSD", "HF"]:
#         raise NotImplementedError("Method %s not supported." % method)

#     deltaZ = -com.nuclear_charges*(1-lam_val)
#     includeonly = np.arange(len(mol.atom_coords()))


#     def add_qmmm(calc, mol, deltaZ):
#         mf = pyscf.qmmm.mm_charge(calc, mol.atom_coords()[includeonly]/ angstrom, deltaZ)

#         def energy_nuc(self):
#             q = mol.atom_charges().astype(np.float)
#             q[includeonly] += deltaZ
#             return mol.energy_nuc(q)

#         mf.energy_nuc = energy_nuc.__get__(mf, mf.__class__)

#         return mf

#     if method == "HF":
#         calc = add_qmmm(pyscf.scf.RHF(mol), mol, deltaZ)
#         hfe = calc.kernel(verbose=0)
#         dm1_ao = calc.make_rdm1()
#         total_energy = calc.e_tot
#     if method == "CCSD":
#         calc = add_qmmm(pyscf.scf.RHF(mol), mol, deltaZ)
#         hfe = calc.kernel(verbose=0)
#         mycc = pyscf.cc.CCSD(calc).run()
#         dm1 = mycc.make_rdm1()
#         dm1_ao = np.einsum("pi,ij,qj->pq", calc.mo_coeff, dm1, calc.mo_coeff.conj())
#         total_energy = mycc.e_tot

#     # GRIDLESS, as things should be ############################
#     # Total energy of SCF run

#     print("TOTAL_ENERGY", total_energy)

#     # Electronic EPN from electron density
#     alch_pot_tmp = []
#     for site in includeonly:
#         mol.set_rinv_orig_(mol.atom_coords()[site])
# #         print("ELECTRONIC_EPN", site, np.matmul(dm1_ao, mol.intor("int1e_rinv")).trace())
#         alch_pot_tmp.append(np.matmul(dm1_ao, mol.intor("int1e_rinv")).trace())
#     alchemical_potentials.append(alch_pot_tmp)
    
#     # energy of HOMO and LUMO
#     nocc = int((total_elec - mol.charge)/2)
#     e_homo.append(calc.mo_energy[nocc-1])
#     e_lumo.append(calc.mo_energy[nocc])